# CPRN CREATE : PHASE A3 : FAC BINDING
**CPRN构建 A3 : 设施绑定**

---

`version` : `4.1.7`   
`author` : seika <seika@live.ca>   
`create_date` : `2025-05-13`   
`last_update` : `2025-11-05`   

---
**A3 阶段功能** ⌼ 设施绑定到基础路网

## ⓿🌞 CONFIGS | 配置项

In [ ]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd

# --- SETTINGS ---

VERSION_VENDOR = 'V417'    # Version Vendor for Output

## EXTENSIONS
PATH_SQLITE_EXT : str = r'/opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib'

## INPUTS
PATH_SQLITE_CPRN : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite'

### 加载FacBind数据表
LAYER_NAME_CPRN_FACPROJ_GTR : str = 'CPRN_facproj_gtr_hdmap_V416_jiangsu'       # 门架投影数据（从前序阶段生成）
LAYER_NAME_CPRN_FACPROJ_SA : str = 'cprn_facproj_sa_dcp_v416_jiangsu'           # 服务区设施投影顶点
LAYER_NAME_CPRN_FACBIND_INTC : str = 'CPRN_facbind_intc_dcp_v417E_jiangsu'      # 枢纽设施绑定

## OUTPUTS
PATH_FOLDER_CPRN_PAYLOAD : str = r'/Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/'
CPRN_MODEL_NAME = fr'CPRN_DG_FAC_EMBD_JS_FUSIONMAP_{VERSION_VENDOR}.pkl'

### 可选：分合流点表
LAYER_NAME_CPRN_CHECK_DCNODES_HDMAP: str = 'CPRN_dg_divcon_nodes_' + VERSION_VENDOR + '_jiangsu'    # 分合流点检查

## ㊀ Load CPRN BASE MODEL

In [2]:
import pandas as pd
from cprn.data.pickle import PickleIO

MODELNAME_CPRN_BASE : str = 'CPRN_DG_BASE_JS_FUSIONMAP_V416_251028_77387ff4e6adc9c4e2a2e831d15c37cdd7a58a51a160f9660ea0b760b127e3f2.tar.gz'

DG_CPRN = PickleIO.load_from_pickle(PATH_FOLDER_CPRN_PAYLOAD + 'CPRN_JS_V416/' + MODELNAME_CPRN_BASE, compress=True)
print(DG_CPRN)

File hash checking : 77387ff4e6adc9c4e2a2e831d15c37cdd7a58a51a160f9660ea0b760b127e3f2 : passed
DiGraph with 219848 nodes and 223773 edges


## ㊁ FAC BIND to CPRN

设施绑定: 
  - 输入1 : CPRN DG
  - 输入2 : 设施数据 (DF/GDF)
  - 输出 : 绑定后的 CPRN DG
  - 过程: 
    - 根据设施数据的几何信息, 在 CPRN DG 中查找最近邻的边
    - 将设施数据绑定到最近邻的边上
    - 返回绑定后的 CPRN DG
    

### ㊁🅰 Gantry Binding
Bind Gantries to CPRN_DG
- input : `CPRN_DG`
- input : Gantry Data (retrieved from DB, table : `CPRN_facproj_gtr_hdmap_V416_jiangsu`)
- output : `DG_CPRN_FAC_EMBD`

In [3]:
## ㊁🅰① Load Gantry Projection Data from Persisted Table
# 从持久化表读取门架投影数据

from taisl_sop.data.base.spatialite import SpatialitePuller

# 初始化 SQLite 连接
sp_cprn = SpatialitePuller(
    path_sqlite=PATH_SQLITE_CPRN, path_spatialite_ext=PATH_SQLITE_EXT,
    set_precision=True, decimal_precision=16, driver='sqlite', verbose=False)

# 构建 SQL 查询
sql_gtr_proj = f"""
    SELECT fac_code, fac_type, fac_name, dist_snap,
           gh_fac, gh_fac_proj, ghz_fac_proj,
           ASText(GEOMETRY) AS wkt
      FROM {LAYER_NAME_CPRN_FACPROJ_GTR};
      """

print(f"📂 Loading GTR-PROJ from layer/table : {LAYER_NAME_CPRN_FACPROJ_GTR}")

# 使用 query_sql_gdf 方法读取数据
gdf_gtr_proj = sp_cprn.query_sql_gdf(sql_gtr_proj, col_geom=['wkt'])
gdf_gtr_proj.rename_geometry('geom', inplace=True)

print(f"✅ Loaded {len(gdf_gtr_proj)} gantry projection records")
print(f"📋 Columns: {list(gdf_gtr_proj.columns)}")

gdf_gtr_proj.sample(min(4, len(gdf_gtr_proj)))

📂 Loading GTR-PROJ from layer/table : CPRN_facproj_gtr_hdmap_V416_jiangsu
✅ Loaded 2590 gantry projection records
📋 Columns: ['fac_code', 'fac_type', 'fac_name', 'dist_snap', 'gh_fac', 'gh_fac_proj', 'ghz_fac_proj', 'wkt', 'geom']


,fac_code,fac_type,fac_name,dist_snap,gh_fac,gh_fac_proj,ghz_fac_proj,wkt,geom
366,G401232001000110010,G1,新昌枢纽至南渡,11.723741,WTSFUMGS40GR,WTSFUMGES781,WTSFUMGES781001149,LINESTRING Z(119.371352 31.453176 11.486428525...,"LINESTRING Z (119.37135 31.45318 11.48643, 119..."
2161,S004832002040620010,G5,宜兴西出口匝道,4.989416,WTT4TUU0JJRZ,WTT4TUSPVJ4J,WTT4TUSPVJ4J001300,LINESTRING Z(119.789654 31.403075 12.997779307...,"LINESTRING Z (119.78965 31.40308 12.99778, 119..."
236,G250332003000720010,G1,南庄至殷巷1,12.814099,WTSMREP176K9,WTSMREP1FZ92,WTSMREP1FZ92001221,LINESTRING Z(118.815911 31.877058 12.208653461...,"LINESTRING Z (118.81591 31.87706 12.20865, 118..."
1466,G152232001040320010,G5,沙家浜出口匝道,5.255471,WTTGKT6RG3YY,WTTGKT6RURVW,WTTGKT6RURVW001229,LINESTRING Z(120.78693 31.538969 12.2871796740...,"LINESTRING Z (120.78693 31.53897 12.28718, 120..."


In [4]:
## PREPARE
cols_facembd = ['fac_code', 'fac_type', 'fac_name', 'gh_fac', 'ghz_fac_proj',] #  'rrl_id', 'osm_id']
gdf_fac_gtr_bind = gdf_gtr_proj[cols_facembd].copy()
gdf_fac_gtr_bind = gdf_fac_gtr_bind.rename(columns={'ghz_fac_proj':'vtx_fac',}) # 'rrl_id':'rrl_id_hdm', 'osm_id':'rrl_id_osm'})
gdf_fac_gtr_bind['fac_ghz'] = gdf_fac_gtr_bind['gh_fac'] + '-00000'
gdf_fac_gtr_bind = gdf_fac_gtr_bind[['vtx_fac', 'fac_code', 'fac_type', 'fac_name', 'fac_ghz',]] #'rrl_id_hdm', 'rrl_id_osm',]]

print(f"gtr to bind : {len(gdf_fac_gtr_bind)}")

## BIND
from taisl_sop.model.cprn.fac_embd import FacEmbedder as FacEmbd
DG_CPRN_FAC_EMBD = FacEmbd.fac_embed_cprn(cprn = DG_CPRN, gdf_fac = gdf_fac_gtr_bind, 
    col_vtx = 'vtx_fac', inplace = False, verbose = True)

gtr to bind : 2590


In [5]:
# check
# gdf_fac_gtr_bind.sample(4)
DG_CPRN_FAC_EMBD.nodes['WTSW3V30BSGD002220']['lst_fac_attr']

[{'vtx_fac': 'WTSW3V30BSGD002220',
  'fac_code': 'G004232001041510010',
  'fac_type': 'G4',
  'fac_name': '南京收费站入口匝道',
  'fac_ghz': 'WTSW3V310MZG-00000'}]

### ㊁🅱 DivConv Binding
- 从 CPRN 提取分合流点
- 将 分合流点(的属性) 作为设施嵌入到 CPRN

In [6]:
### EXTRACT DC NODE

import pandas as pd
from taisl_sop.model.topo_nx import TopoNx

dg_cprn = DG_CPRN_FAC_EMBD.copy()
dct_dc_nodes = TopoNx.find_divergent_convergent_nodes(dg_cprn)  #TODO: 需要优化

df_dc_nodes = pd.DataFrame(dct_dc_nodes).T.drop(
    columns=['is_divergent', 'is_convergent', 
             'in_edges_attr', 'out_edges_attr', 'node_type'])

def get_dc_type(in_degree, out_degree):
    if in_degree > 1 and out_degree == 1:
        return 'CONV'
    elif in_degree == 1 and out_degree > 1:
        return 'DIV'
    elif in_degree >1 and out_degree > 1:
        return 'MULTI'
    else:
        return 'OTHER'

## make df of dcp fac-like (for binding)

df_dc_nodes['dc_type'] = df_dc_nodes.apply(
    lambda row: get_dc_type(row['in_degree'], row['out_degree']), axis=1)
df_dc_nodes['gh_dcp'] = df_dc_nodes.index

dct_col_rename_factype = {'CONV': 'DC1', 'DIV': 'DC2', 'MULTI': 'DC3', 'OTHER': 'DCX'}
df_dc_nodes['fac_type'] = df_dc_nodes['dc_type'].map(dct_col_rename_factype)
df_dc_nodes = df_dc_nodes.query("dc_type != 'OTHER'")   # ⚠️ 其他类型不嵌入, e.g., 0->2, 3->0

df_dc_nodes['vtx_fac'] = df_dc_nodes['gh_dcp']
df_dc_nodes['fac_code'] = df_dc_nodes['gh_dcp']
df_dc_nodes['fac_name'] = df_dc_nodes['in_degree'].astype(str) + '|' + df_dc_nodes['out_degree'].astype(str)

from taisl_sop.model.geohash import Geohash
df_dc_nodes['geom_fac_wgs_3d'] = Geohash.decode_df_gdf(df_dc_nodes, colname_geohash='gh_dcp', decode_z=True)['geometry']
df_dc_nodes['geom_fac_wgs'] = Geohash.decode_df_gdf(df_dc_nodes, colname_geohash='gh_dcp', decode_z=False)['geometry']

print(len(df_dc_nodes))
df_dc_nodes.sample(4)

9113


,in_degree,out_degree,dc_type,gh_dcp,fac_type,vtx_fac,fac_code,fac_name,geom_fac_wgs_3d,geom_fac_wgs
WTTK0P2ENF9H001582,1,2,DIV,WTTK0P2ENF9H001582,DC2,WTTK0P2ENF9H001582,WTTK0P2ENF9H001582,1|2,POINT Z (119.88379 31.68098 15.82),POINT (119.88379 31.68098)
WTTYDQE7P03R000000,1,2,DIV,WTTYDQE7P03R000000,DC2,WTTYDQE7P03R000000,WTTYDQE7P03R000000,1|2,POINT Z (120.68958 32.1163 0),POINT (120.68958 32.1163)
WTSTC36HVE46002314,2,1,CONV,WTSTC36HVE46002314,DC1,WTSTC36HVE46002314,WTSTC36HVE46002314,2|1,POINT Z (118.88604 31.95594 23.14),POINT (118.88604 31.95594)
WTSKNP7T7G02003165,2,1,CONV,WTSKNP7T7G02003165,DC1,WTSKNP7T7G02003165,WTSKNP7T7G02003165,2|1,POINT Z (118.7452 31.68137 31.65),POINT (118.7452 31.68137)


In [7]:
## BIND DCP to CPRN
cols_fac_embd_dcp = ['vtx_fac', 'fac_code', 'fac_type', 'fac_name',]
df_fac_dcp_bind = df_dc_nodes[cols_fac_embd_dcp]

from taisl_sop.model.cprn.fac_embd import FacEmbedder as FacEmbd
DG_CPRN_FAC_EMBD = FacEmbd.fac_embed_cprn(cprn = DG_CPRN_FAC_EMBD, gdf_fac = df_fac_dcp_bind, 
    col_vtx = 'vtx_fac', inplace = False, verbose = True)

#### 🌖 (支线任务) 分/合流点 导出地图数据检查

In [8]:
subtask_export_dcp_map = False

if subtask_export_dcp_map:
    cols_fac_dcp = ['fac_code', 'fac_type', 'fac_name', 'vtx_fac', 'geom_fac_wgs', 'geom_fac_wgs_3d',]
    gdf_dc_nodes = gpd.GeoDataFrame(df_dc_nodes[cols_fac_dcp])
    gdf_dc_nodes = gdf_dc_nodes.set_geometry('geom_fac_wgs_3d')
    gdf_dc_nodes = gdf_dc_nodes.drop(columns='geom_fac_wgs')

    print(LAYER_NAME_CPRN_CHECK_DCNODES_HDMAP)

    #🌖 EXPORT TO DB
    gdf_dc_nodes.to_file(PATH_SQLITE_CPRN, driver='SQLite', spatialite=True, 
                         layer=LAYER_NAME_CPRN_CHECK_DCNODES_HDMAP, overwrite=True,
                         index=False, mode='w', encoding='utf-8')

### ㊁🅲 SA(DC) Binding
从 DB 中获取 SA_DCP 数据, 由过程 B1 `PHASE_B1_FAC_PROC_SA.ipynb` 生成


In [9]:
# Fetch SA_DCP data from DB, check `PHASE_B1_FAC_PROC_SA_V416.ipynb` for how to generate it.
from taisl_sop.data.cprn.fac.fac_bind_sa import FacBindSaFetcher
sp_cprn = SpatialitePuller(path_sqlite=PATH_SQLITE_CPRN, path_spatialite_ext=PATH_SQLITE_EXT, 
    set_precision=True, decimal_precision=16, driver='sqlite', verbose=False)

fac_bind_sa_fetcher = FacBindSaFetcher(spatialite_puller=sp_cprn, 
    tb_name=LAYER_NAME_CPRN_FACPROJ_SA, verbose=False)

gdf_fac_sa_dcp = fac_bind_sa_fetcher.fetch_fac_bind_sa().drop(columns=['wkt', 'fac_bind_geohash', 'fac_geohash'])
gdf_fac_sa_dcp = gdf_fac_sa_dcp.rename(columns={'fac_vtx':'vtx_fac'})

print (f"SA_DCP to bind : {len(gdf_fac_sa_dcp)}")

SA_DCP to bind : 482


In [10]:
# BIND SA_DCP to CPRN
cols_fac_sa_dcp = ['vtx_fac', 'fac_code', 'fac_type', 'fac_name', 'fac_ghz']
gdf_fac_sa_dcp_bind = gdf_fac_sa_dcp[cols_fac_sa_dcp]

from taisl_sop.model.cprn.fac_embd import FacEmbedder as FacEmbd
DG_CPRN_FAC_EMBD = FacEmbd.fac_embed_cprn(cprn = DG_CPRN_FAC_EMBD, gdf_fac = gdf_fac_sa_dcp_bind, 
    col_vtx = 'vtx_fac', inplace = False, verbose = True)

print(DG_CPRN_FAC_EMBD)

DiGraph with 219848 nodes and 223773 edges


### ㊁🄳 枢纽绑定 (实现1 /Eco Ver)

In [11]:
## ㊁🄳① Load Interchange Binding Data from Persisted Table
# 从持久化表读取枢纽绑定数据

from taisl_sop.data.base.spatialite import SpatialitePuller

# 初始化 SQLite 连接（如需要可重用前面的 sp_cprn）
sp_cprn = SpatialitePuller(path_sqlite=PATH_SQLITE_CPRN, path_spatialite_ext=PATH_SQLITE_EXT,
    set_precision=True, decimal_precision=16, driver='sqlite', verbose=False)

# 构建 SQL 查询
sql_intc_bind = f"""
    SELECT fac_code, fac_type, fac_name, fac_vtx, 
           fac_geohash, road_code, mdir, dist_cp, fac_rp_src,
           ASText(GEOMETRY) AS wkt
      FROM {LAYER_NAME_CPRN_FACBIND_INTC};
      """

print(f"📂 Loading INTC-BIND from layer/table : {LAYER_NAME_CPRN_FACBIND_INTC}")

# 使用 query_sql_gdf 方法读取数据
gdf_intc_bind = sp_cprn.query_sql_gdf(sql_intc_bind, col_geom=['wkt'])
gdf_intc_bind.rename_geometry('geom', inplace=True)

print(f"✅ Loaded {len(gdf_intc_bind)} interchange binding records")
print(f"📋 Columns: {list(gdf_intc_bind.columns)}")
print(f"📊 Facility type distribution in retrieved data:")
print(gdf_intc_bind['fac_type'].value_counts())

# gdf_intc_bind.sample(min(4, len(gdf_intc_bind)))

📂 Loading INTC-BIND from layer/table : CPRN_facbind_intc_dcp_v417E_jiangsu
✅ Loaded 1566 interchange binding records
📋 Columns: ['fac_code', 'fac_type', 'fac_name', 'fac_vtx', 'fac_geohash', 'road_code', 'mdir', 'dist_cp', 'fac_rp_src', 'wkt', 'geom']
📊 Facility type distribution in retrieved data:
fac_type
IC2E    780
IC1E    779
Name: count, dtype: int64


In [12]:
gdf_intc_bind.sample(4)

,fac_code,fac_type,fac_name,fac_vtx,fac_geohash,road_code,mdir,dist_cp,fac_rp_src,wkt,geom
325,G0042K03110000LC,IC1E,刘村枢纽,WTSMSX8XFWY2001140,WTSMSRNPM2EF,,FWD,446.644710,algo,"LINESTRING Z(118.6718 31.943997 0, 118.6751188...","LINESTRING Z (118.6718 31.944 0, 118.67512 31...."
1147,S0048K01360000XY,IC1E,雪堰枢纽,WTT76TR4GEYJ001426,WTT76VY9QJHR,S48,BWD,1026.081040,algo,"LINESTRING Z(120.012852 31.540617 0, 120.00244...","LINESTRING Z (120.01285 31.54062 0, 120.00245 ..."
1098,G4221K0255000LJB,IC1E,骆家边枢纽,WTSS76ZYY6EZ002641,WTSS7E0BMHCG,G4221,FWD,128.409232,algo,"LINESTRING Z(118.983181 31.701119 0, 118.98186...","LINESTRING Z (118.98318 31.70112 0, 118.98186 ..."
365,G0042K01850000LX,IC1E,罗溪枢纽,WTTJRGBJR0BR001584,WTTJRUCJ1CC2,S39,BWD,614.520178,algo,"LINESTRING Z(119.873276 31.887312 0, 119.87212...","LINESTRING Z (119.87328 31.88731 0, 119.87213 ..."


In [13]:
## ㊁🄳② Prepare INTC Binding Data for Embedding
# 标准化枢纽绑定数据格式

# 选择需要的列
cols_intc_bind = ['fac_code', 'fac_type', 'fac_name', 'fac_vtx', 'fac_geohash', 'road_code', 'mdir', 'dist_cp', 'fac_rp_src']
gdf_fac_intc_bind = gdf_intc_bind[cols_intc_bind].copy()

# 重命名列以符合 fac_embed_cprn 的标准格式
gdf_fac_intc_bind = gdf_fac_intc_bind.rename(columns={'fac_vtx': 'vtx_fac'})

# 将 fac_geohash 转换为 fac_ghz 格式（添加高程后缀，保持与其他设施一致）
gdf_fac_intc_bind['fac_ghz'] = gdf_fac_intc_bind['fac_geohash'] + '-00000'

gdf_fac_intc_bind = gdf_fac_intc_bind.fillna(value={'road_code': 'UNK', 'mdir': 'UNK'})
gdf_fac_intc_bind['road_code'] = gdf_fac_intc_bind['road_code'].replace('', 'UNK')

# 删除原始的 fac_geohash 列，重新排列列顺序
gdf_fac_intc_bind = gdf_fac_intc_bind[['vtx_fac', 'fac_code', 'fac_type', 'fac_name', 'fac_ghz', 'road_code', 'mdir', 'dist_cp', 'fac_rp_src']]

print(f"✅ 标准化枢纽绑定数据: {len(gdf_fac_intc_bind)} 条")
print(f"📋 标准化后的列: {list(gdf_fac_intc_bind.columns)}")
print(f"📊 设施类型分布:")
print(gdf_fac_intc_bind['fac_type'].value_counts())

gdf_fac_intc_bind.sample(min(4, len(gdf_fac_intc_bind)))

✅ 标准化枢纽绑定数据: 1566 条
📋 标准化后的列: ['vtx_fac', 'fac_code', 'fac_type', 'fac_name', 'fac_ghz', 'road_code', 'mdir', 'dist_cp', 'fac_rp_src']
📊 设施类型分布:
fac_type
IC2E    780
IC1E    779
Name: count, dtype: int64


,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,road_code,mdir,dist_cp,fac_rp_src
753,WTTBT5KMZY7N002052,G0050K00690000FH,IC2E,汾湖枢纽,WTTBT5S3UV25-00000,UNK,BWD,76.565043,algo
112,WTTDYJDHTX4F001438,G0002K11220000DQ,IC1E,东桥枢纽,WTTDYHVB68VD-00000,G2|G42,FWD,722.013729,algo
1129,WTTDT44KBJ49001196,S0017K00320000TA,IC2E,通安枢纽,WTTDSFWSEXB4-00000,S58,BWD,563.269357,algo
1312,WTWPT2PNRR9U001802,S0015K0022000RDN,IC1E,如东南枢纽,WTWPMRKS3SKG-00000,S15,FWD,605.815385,algo


In [14]:
gdf_fac_intc_bind.sample(min(4, len(gdf_fac_intc_bind)))

,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,road_code,mdir,dist_cp,fac_rp_src
287,WTSMYSQ1G1ZY002042,G0042K0300000HSM,IC1E,花神庙枢纽,WTSMYUE04HGF-00000,G42|G312|S001,BWD,655.742572,algo
97,WTTDY7BWJQW2001386,G0002K11220000DQ,IC1E,东桥枢纽,WTTDYHVB68VD-00000,G2|G42,BWD,619.175128,algo
970,WW56W01KQ0SR004141,G0030K02140000LD,IC2E,林东枢纽,WW56W100JQD0-00000,G2513,BWD,556.194752,algo
489,WWHU905QSW6V000710,G0025K17060000NH,IC1E,宁海枢纽,WWHU3PG11WG2-00000,G25,BWD,260.548769,algo


In [15]:
## ㊁🄳③ Bind INTC to CPRN
# 绑定枢纽设施到 CPRN

from taisl_sop.model.cprn.fac_embd import FacEmbedder as FacEmbd
DG_CPRN_FAC_EMBD = FacEmbd.fac_embed_cprn(
    cprn=DG_CPRN_FAC_EMBD, 
    gdf_fac=gdf_fac_intc_bind, 
    col_vtx='vtx_fac', 
    inplace=False, 
    verbose=True)

print(f"✅ 枢纽设施绑定完成")
print(DG_CPRN_FAC_EMBD)

✅ 枢纽设施绑定完成
DiGraph with 219848 nodes and 223773 edges


### 🅇 从CPRN取回BindedFAC检查

In [16]:
from cprn.model.topo.topo_search import CprnTopoSearch as CTS
df = CTS.list_fac_df(DG_CPRN_FAC_EMBD)

print(f" fetch embede fac from CPRN: {len(df)}" )
df.sample(4) 

 fetch embede fac from CPRN: 13751


,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,road_code,mdir,dist_cp,fac_rp_src
4220,WTTF2GNWSDN5001251,G0002K1142000SZB,IC2E,苏州北枢纽,WTTF36G16MVN-00000,UNK,BWD,1643.36304,algo
4863,WTU9CMECTWQD003013,WTU9CMECTWQD003013,DC1,2|1,NaN,NaN,NaN,NaN,NaN
6432,WWHTXT9PJV66000991,G002532001000520010,G1,大浦至宋庄1,WWHTXT9PWG84-00000,NaN,NaN,NaN,NaN
12092,WTTJGPEXZSY2001389,G004232001001220010,G1,丹阳新区枢纽至丹阳东1,WTTJGPEXK949-00000,NaN,NaN,NaN,NaN


In [ ]:
df.query(" fac_name == '' ")

,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,road_code,mdir,dist_cp,fac_rp_src
2118,WTSX009UVD3P001942,G0036A1000BGZ0E,SA1,八卦洲服务区(东),WTSX012JSYMW-00000,NaN,NaN,NaN,NaN
2710,WTSRPCYPSBYR000960,G0036A1000BGZ0E,SA2,八卦洲服务区(东),WTSX012JSYMW-00000,NaN,NaN,NaN,NaN


## ㊂ PERSISTENCE 1: 设施嵌 CPRN 持久化 (PICKLE)

In [19]:
from cprn.data.pickle import PickleIO

CPRN_MODEL_NAME = r'CPRN_DG_FAC_EMBD_JS_FUSIONMAP_V417.pkl'
PickleIO.dump_as_pickle(DG_CPRN_FAC_EMBD, PATH_FOLDER_CPRN_PAYLOAD + CPRN_MODEL_NAME, compress=True)

File renamed from /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_FAC_EMBD_JS_FUSIONMAP_V417_251105_temp.tar.gz to /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_FAC_EMBD_JS_FUSIONMAP_V417_251105_baaac0479ee7a4ea48b18a07d053cabc754eea0a5f7f696bc5a7437f50ec7e00.tar.gz
Compressed pickle file saved as: /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_FAC_EMBD_JS_FUSIONMAP_V417_251105_baaac0479ee7a4ea48b18a07d053cabc754eea0a5f7f696bc5a7437f50ec7e00.tar.gz


------

In [20]:
print("EOF")

EOF
